# NB Regression with PyTorch

Negative binomial regression is in fact a 1-layer neural network with a special loss function. Here we increase the number of layers so that the mean $\mu$ of the negative binomial distribution is no longer approximated by a purely linear combination of the features. At the same time, we also regress the dispersion $\alpha$ which determines the variance ($\mu+\alpha\mu²$).

In [2]:
from NBPyTorch import NBNet, PoNet, NBNLLLoss
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import numpy as np
import scipy as sc
from util import MyUtil

%load_ext autoreload
%autoreload 2

util=MyUtil()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def to_numpy(muA,alphaA,muB,alphaB):
    return muA.data.numpy(),alphaA.data.numpy(),muB.data.numpy(),alphaB.data.numpy()

# Encode gametype
rounds = {
    "Finale": 1,
    "Spiel um Platz Drei": 2,
    "Halbfinale": 3,
    "Viertelfinale": 4,
    "Achtelfinale": 5,
    "Gruppenphase": 6
}
def map_to_round(x):
    if x.startswith("Gruppe"):
        return rounds["Gruppenphase"]
    else:
        return rounds[x]

def scale(data,state):
    data["teamA_def_val"]=(data["teamA_def_val"]-state["teamA_def_mean"])/state["teamA_def_std"]
    data["teamA_off_val"]=(data["teamA_off_val"]-state["teamA_off_mean"])/state["teamA_off_std"]
    data["teamB_def_val"]=(data["teamB_def_val"]-state["teamB_def_mean"])/state["teamB_def_std"]
    data["teamB_off_val"]=(data["teamB_off_val"]-state["teamB_off_mean"])/state["teamB_off_std"]
    data["teamA_frag"]=(data["teamA_frag"]-state["teamA_frag_mean"])/state["teamA_frag_std"]
    data["teamB_frag"]=(data["teamB_frag"]-state["teamB_frag_mean"])/state["teamB_frag_std"]
    data["teamA_age"]=(data["teamA_age"]-state["teamA_age_mean"])/state["teamA_age_std"]
    data["teamB_age"]=(data["teamB_age"]-state["teamB_age_mean"])/state["teamB_age_std"]

In [ ]:
# data/final.csv: past tournaments
#data = pd.read_csv(filepath_or_buffer="data/final.csv",delimiter=";",index_col=False).round(2)
# data/final_w_18: updated with WC 2018 results as the tournament progressed
data = pd.read_csv(filepath_or_buffer="data/final_w_18.csv",delimiter=";",index_col=False).round(2)

# impute missing past values with 0
data.fillna({"past_resultA":0,"past_resultB":0},inplace=True)
# drop id columns
data.drop(["gameid","teamidA","teamidB"],axis=1,inplace=True)

# Encode gametype
rounds = {
    "Finale": 1,
    "Spiel um Platz Drei": 2,
    "Halbfinale": 3,
    "Viertelfinale": 4,
    "Achtelfinale": 5,
    "Gruppenphase": 6
}
def map_to_round(x):
    if x.startswith("Gruppe"):
        return rounds["Gruppenphase"]
    else:
        return rounds[x]

data["gametype"]=data["gametype"].apply(map_to_round)
#data.max()

In [ ]:
# knockout stage w/o pens
data_ko=data.drop(data[data.gametype == 6].index,axis=0).copy()
data_ko.drop(data_ko[data_ko.addinfo == 'n.E.'].index,axis=0,inplace=True) # drop pens

# group stage 
data_gr=data.drop(data[data.gametype != 6].index,axis=0)

## Knockout stage

In [ ]:
data_ko.describe()

In [ ]:
data_train, data_test = train_test_split(data_ko.copy(),test_size=0.2)
col=["gametype","teamA_age","teamB_age","teamA_def_val","teamB_def_val","teamA_off_val","teamB_off_val","teamA_frag","teamB_frag","past_resultA","past_resultB"]

In [ ]:
teamA_def_mean = data_train["teamA_def_val"].mean()
teamA_def_std = data_train["teamA_def_val"].std()
teamA_off_mean = data_train["teamA_off_val"].mean()
teamA_off_std = data_train["teamA_off_val"].std()

teamB_def_mean = data_train["teamB_def_val"].mean()
teamB_def_std = data_train["teamB_def_val"].std()
teamB_off_mean = data_train["teamB_off_val"].mean()
teamB_off_std = data_train["teamB_off_val"].std()

teamA_frag_mean = data_train["teamA_frag"].mean()
teamA_frag_std = data_train["teamA_frag"].std()

teamB_frag_mean = data_train["teamB_frag"].mean()
teamB_frag_std = data_train["teamB_frag"].std()

teamA_age_mean = data_train["teamA_age"].mean()
teamA_age_std = data_train["teamA_age"].std()

teamB_age_mean = data_train["teamB_age"].mean()
teamB_age_std = data_train["teamB_age"].std()

In [ ]:
# scale train features
data_train["teamA_def_val"]=(data_train["teamA_def_val"]-teamA_def_mean)/teamA_def_std
data_train["teamA_off_val"]=(data_train["teamA_off_val"]-teamA_off_mean)/teamA_off_std
data_train["teamB_def_val"]=(data_train["teamB_def_val"]-teamB_def_mean)/teamB_def_std
data_train["teamB_off_val"]=(data_train["teamB_off_val"]-teamB_off_mean)/teamB_off_std
data_train["teamA_frag"]=(data_train["teamA_frag"]-teamA_frag_mean)/teamA_frag_std
data_train["teamB_frag"]=(data_train["teamB_frag"]-teamB_frag_mean)/teamB_frag_std
data_train["teamA_age"]=(data_train["teamA_age"]-teamA_age_mean)/teamA_age_std
data_train["teamB_age"]=(data_train["teamB_age"]-teamB_age_mean)/teamB_age_std

# scale test features
data_test["teamA_def_val"]=(data_test["teamA_def_val"]-teamA_def_mean)/teamA_def_std
data_test["teamA_off_val"]=(data_test["teamA_off_val"]-teamA_off_mean)/teamA_off_std
data_test["teamB_def_val"]=(data_test["teamB_def_val"]-teamB_def_mean)/teamB_def_std
data_test["teamB_off_val"]=(data_test["teamB_off_val"]-teamB_off_mean)/teamB_off_std
data_test["teamA_frag"]=(data_test["teamA_frag"]-teamA_frag_mean)/teamA_frag_std
data_test["teamB_frag"]=(data_test["teamB_frag"]-teamB_frag_mean)/teamB_frag_std
data_test["teamA_age"]=(data_test["teamA_age"]-teamA_age_mean)/teamA_age_std
data_test["teamB_age"]=(data_test["teamB_age"]-teamB_age_mean)/teamB_age_std

In [ ]:
data_train.describe()

In [ ]:
X_train = data_train[col].values
y_train = data_train[["resultA","resultB"]].values
X_test = data_test[col].values
y_test = data_test[["resultA","resultB"]].values

In [ ]:
X_tr = torch.from_numpy(X_train).float()
y_trA = torch.from_numpy(y_train[:,[0]]).float()
y_trB = torch.from_numpy(y_train[:,[1]]).float()
X_te = torch.from_numpy(X_test).float()

### Negative Binomial distribution

#### Model

In [ ]:
crit = NBNLLLoss(eps=1e-3)
def fn_print(module, grad_input, grad_output):
    print("Gradients Input",grad_input)
    print("Gradients Output", grad_output)
    return None

In [ ]:
neural = NBNet(len(col),30,20,4,0.3)
optimizer = optim.RMSprop(params=neural.parameters(), lr=1e-3, alpha=0.99, eps=1e-05, weight_decay=0, momentum=0, centered=False)

In [ ]:
for epoch in range(200):
    # train
    neural.train()
    muA,alphaA,muB,alphaB = neural(X_tr)
    lossA = crit(muA,alphaA,y_trA)
    lossB = crit(muB,alphaB,y_trB)
    loss = lossA+lossB
    muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
    tend_acc_tr = util.tend_acc_nb(muA,alphaA,muB,alphaB,y_train)
    # update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # evaluate
    neural.eval()
    muA,alphaA,muB,alphaB = neural(X_te)
    muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
    tend_acc_val = util.tend_acc_nb(muA,alphaA,muB,alphaB,y_test)
    print(epoch,loss.item(),tend_acc_tr,tend_acc_val)


#### Validation

In [ ]:
neural.eval()
muA,alphaA,muB,alphaB=neural(X_te)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)

In [ ]:
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y=y_test,y_prob=probs,top_n=5,verbose=True)

In [ ]:
state = {
    "teamA_def_mean": teamA_def_mean,
    "teamA_def_std": teamA_def_std,
    "teamA_off_mean": teamA_off_mean,
    "teamA_off_std": teamA_off_std,
    "teamB_def_mean": teamB_def_mean,
    "teamB_def_std": teamB_def_std,
    "teamB_off_mean": teamB_off_mean,
    "teamB_off_std": teamB_off_std,
    "teamA_frag_mean": teamA_frag_mean, 
    "teamA_frag_std": teamA_frag_std,
    "teamB_frag_mean": teamB_frag_mean, 
    "teamB_frag_std": teamB_frag_std,
    "teamA_age_mean": teamA_age_mean, 
    "teamA_age_std": teamA_age_std,
    "teamB_age_mean": teamB_age_mean,
    "teamB_age_std": teamB_age_std,
    "state_dict": neural.state_dict(),
}
#torch.save(state, 'model/model_ko.pth')

### Poisson distribution

In [ ]:
crit_pois = torch.nn.PoissonNLLLoss()
neural_pois = PoNet(len(col),30,20,2,0.2)
optimizer_pois = optim.RMSprop(params=neural.parameters(), lr=1e-3, alpha=0.99, eps=1e-05, weight_decay=0, momentum=0, centered=False)

In [ ]:
for epoch in range(200):
    # train
    neural_pois.train()
    mu = neural_pois(X_tr)
    lossA_pois = crit_pois(mu[:,[0]],y_trA)
    lossB_pois = crit_pois(mu[:,[1]],y_trB)
    loss_pois = lossA_pois+lossB_pois    
    tend_acc_tr = util.tend_acc_pois(mu.data.numpy(),y_train)
    # evaluate
    neural_pois.eval()
    mu = neural_pois(X_te)
    tend_acc_val = util.tend_acc_pois(mu.data.numpy(),y_test)
    print(epoch, loss_pois.item(),tend_acc_tr,tend_acc_val)
    # update
    optimizer_pois.zero_grad()
    loss_pois.backward()
    optimizer_pois.step()


In [ ]:
neural_pois.eval()
mu=neural_pois(X_te).data.numpy()

In [ ]:
probs=util.calc_pois_probs(mu[:,0],mu[:,1])
util.multi_result(y=y_test,y_prob=probs,top_n=5,verbose=True)

### Results
Poisson regression suffers from the same problem as negative binomial regression with fixed dispersion $\alpha$ (it seems to memorize the most frequent outcomes) 

## Group Stage

In [ ]:
data_train, data_test = train_test_split(data_gr.copy(),test_size=0.2)

In [ ]:
teamA_def_mean = data_train["teamA_def_val"].mean()
teamA_def_std = data_train["teamA_def_val"].std()
teamA_off_mean = data_train["teamA_off_val"].mean()
teamA_off_std = data_train["teamA_off_val"].std()

teamB_def_mean = data_train["teamB_def_val"].mean()
teamB_def_std = data_train["teamB_def_val"].std()
teamB_off_mean = data_train["teamB_off_val"].mean()
teamB_off_std = data_train["teamB_off_val"].std()

teamA_frag_mean = data_train["teamA_frag"].mean()
teamA_frag_std = data_train["teamA_frag"].std()

teamB_frag_mean = data_train["teamB_frag"].mean()
teamB_frag_std = data_train["teamB_frag"].std()

teamA_age_mean = data_train["teamA_age"].mean()
teamA_age_std = data_train["teamA_age"].std()

teamB_age_mean = data_train["teamB_age"].mean()
teamB_age_std = data_train["teamB_age"].std()

In [ ]:
# scale some features
data_train["teamA_def_val"]=(data_train["teamA_def_val"]-teamA_def_mean)/teamA_def_std
data_train["teamA_off_val"]=(data_train["teamA_off_val"]-teamA_off_mean)/teamA_off_std
data_train["teamB_def_val"]=(data_train["teamB_def_val"]-teamB_def_mean)/teamB_def_std
data_train["teamB_off_val"]=(data_train["teamB_off_val"]-teamB_off_mean)/teamB_off_std
data_train["teamA_frag"]=(data_train["teamA_frag"]-teamA_frag_mean)/teamA_frag_std
data_train["teamB_frag"]=(data_train["teamB_frag"]-teamB_frag_mean)/teamB_frag_std
data_train["teamA_age"]=(data_train["teamA_age"]-teamA_age_mean)/teamA_age_std
data_train["teamB_age"]=(data_train["teamB_age"]-teamB_age_mean)/teamB_age_std

# scale test features
data_test["teamA_def_val"]=(data_test["teamA_def_val"]-teamA_def_mean)/teamA_def_std
data_test["teamA_off_val"]=(data_test["teamA_off_val"]-teamA_off_mean)/teamA_off_std
data_test["teamB_def_val"]=(data_test["teamB_def_val"]-teamB_def_mean)/teamB_def_std
data_test["teamB_off_val"]=(data_test["teamB_off_val"]-teamB_off_mean)/teamB_off_std
data_test["teamA_frag"]=(data_test["teamA_frag"]-teamA_frag_mean)/teamA_frag_std
data_test["teamB_frag"]=(data_test["teamB_frag"]-teamB_frag_mean)/teamB_frag_std
data_test["teamA_age"]=(data_test["teamA_age"]-teamA_age_mean)/teamA_age_std
data_test["teamB_age"]=(data_test["teamB_age"]-teamB_age_mean)/teamB_age_std

In [ ]:
data_train.describe()

In [ ]:
col=["teamA_age","teamB_age","teamA_def_val","teamB_def_val","teamA_off_val","teamB_off_val","teamA_frag","teamB_frag","past_resultA","past_resultB"]

X_train = data_train[col].values
y_train = data_train[["resultA","resultB"]].values
X_test = data_test[col].values
y_test = data_test[["resultA","resultB"]].values

### Negative Binomial Regression

In [ ]:
X_train = data_train[col].values
y_train = data_train[["resultA","resultB"]].values
X_test = data_test[col].values
y_test = data_test[["resultA","resultB"]].values

X_tr = torch.from_numpy(X_train).float()
y_trA = torch.from_numpy(y_train[:,[0]]).float()
y_trB = torch.from_numpy(y_train[:,[1]]).float()
X_te = torch.from_numpy(X_test).float()

In [ ]:
crit = NBNLLLoss(eps=1e-4)
neural = NBNet(len(col),30,20,4,0.3)
optimizer = optim.RMSprop(params=neural.parameters(), lr=1e-3, alpha=0.99, eps=1e-05, weight_decay=0, momentum=0, centered=False)

In [ ]:
for epoch in range(200):
    # train
    neural.train()
    muA,alphaA,muB,alphaB = neural(X_tr)
    lossA = crit(muA,alphaA,y_trA)
    lossB = crit(muB,alphaB,y_trB)
    loss = lossA+lossB
    muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
    tend_acc_tr = util.tend_acc_nb(muA,alphaA,muB,alphaB,y_train)
    # evaluate
    neural.eval()
    muA,alphaA,muB,alphaB = neural(X_te)
    muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
    tend_acc_val = util.tend_acc_nb(muA,alphaA,muB,alphaB,y_test)
    # update
    print(epoch,loss.item(),tend_acc_tr,tend_acc_val)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
neural.eval()
muA,alphaA,muB,alphaB=neural(X_te)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)

In [ ]:
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y=y_test,y_prob=probs,top_n=1,verbose=True)

In [ ]:
state = {
    "teamA_def_mean": teamA_def_mean,
    "teamA_def_std": teamA_def_std,
    "teamA_off_mean": teamA_off_mean,
    "teamA_off_std": teamA_off_std,
    "teamB_def_mean": teamB_def_mean,
    "teamB_def_std": teamB_def_std,
    "teamB_off_mean": teamB_off_mean,
    "teamB_off_std": teamB_off_std,
    "teamA_frag_mean": teamA_frag_mean, 
    "teamA_frag_std": teamA_frag_std,
    "teamB_frag_mean": teamB_frag_mean, 
    "teamB_frag_std": teamB_frag_std,
    "teamA_age_mean": teamA_age_mean, 
    "teamA_age_std": teamA_age_std,
    "teamB_age_mean": teamB_age_mean,
    "teamB_age_std": teamB_age_std,
    "state_dict": neural.state_dict(),
}
#torch.save(state, 'model/model_r.pth')

### Poisson Regression

In [ ]:
crit_pois = torch.nn.PoissonNLLLoss()
neural_pois = PoNet(len(col),30,20,2,0.3)
optimizer_pois = optim.RMSprop(params=neural.parameters(), lr=1e-3, alpha=0.99, eps=1e-05, weight_decay=0, momentum=0, centered=False)

In [ ]:
for epoch in range(200):
    # train
    neural_pois.train()
    mu = neural_pois(X_tr)
    lossA_pois = crit_pois(mu[:,[0]],y_trA)
    lossB_pois = crit_pois(mu[:,[1]],y_trB)
    loss_pois = lossA_pois+lossB_pois    
    tend_acc_tr = util.tend_acc_pois(mu.data.numpy(),y_train)
    # evaluate
    neural_pois.eval()
    mu = neural_pois(X_te)
    tend_acc_val = util.tend_acc_pois(mu.data.numpy(),y_test)
    print(epoch, loss_pois.item(),tend_acc_tr,tend_acc_val)
    # update
    optimizer_pois.zero_grad()
    loss_pois.backward()
    optimizer_pois.step()

In [ ]:
neural_pois.eval()
mu=neural_pois(X_te).data.numpy()

In [ ]:
probs=util.calc_pois_probs(mu[:,0],mu[:,1])
util.multi_result(y=y_test,y_prob=probs,top_n=5,verbose=True)

# Inference

In [ ]:
#https://pytorch.org/docs/master/notes/serialization.html#best-practices
#https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch

### Pickled

In [ ]:
col=["teamA_age","teamB_age","teamA_def_val","teamB_def_val","teamA_off_val","teamB_off_val","teamA_frag","teamB_frag","past_resultA","past_resultB"]

#### Game 1

In [ ]:
# 1. Spieltag
neural_pickled = NBNet(len(col),20,20,4,0.2)
state = torch.load('model/model_r1.pth')
neural_pickled.load_state_dict(state["state_dict"])
#state

In [ ]:
wm = pd.read_csv(filepath_or_buffer="data/WM18.csv",delimiter=";",index_col=False).round(2)[0:16]
# impute missing past values with 0
wm.fillna({"past_resultA":0,"past_resultB":0},inplace=True)
wm.describe()
#scale
scale(wm, state)

In [ ]:
#wm.describe()

In [ ]:
wm_tr = torch.from_numpy(wm[col].values).float()
neural_pickled.eval()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y_prob=probs,top_n=1,verbose=True)

Top-5 accuracy of 56.25%

Top-3 accuracy of 31.25%

Top-1 accuracy of 18.75%


In [ ]:
util.multi_result(y_prob=probs,y=wm[["resultA","resultB"]].values,top_n=1,verbose=True)

In [ ]:
util.single_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)

In [ ]:
util.multi_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)

#### Game 2

In [ ]:
# 2. Spieltag
# Was retrained on past tournaments and Game 1 results
neural_pickled = NBNet(len(col),30,20,4,0.3)
state = torch.load('model/model_r2.pth')
neural_pickled.load_state_dict(state["state_dict"])
#state

Process World Cup Matches

In [ ]:
wm = pd.read_csv(filepath_or_buffer="data/WM18.csv",delimiter=";",index_col=False).round(2)[16:32]

In [ ]:
# impute missing past values with 0
wm.fillna({"past_resultA":0,"past_resultB":0},inplace=True)

In [ ]:
#scale
scale(wm,state)

In [ ]:
wm_tr = torch.from_numpy(wm[col].values).float()

In [ ]:
neural_pickled.eval()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y_prob=probs,top_n=1,verbose=True)

Top-5 accuracy of 50%

Top-3 accuracy of 37.5%

Top-1 accuracy of 12.50%

In [ ]:
util.multi_result(y_prob=probs,y=wm[["resultA","resultB"]].values,top_n=1,verbose=True)

In [ ]:
util.single_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)

In [ ]:
util.multi_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)

In [ ]:
#wm_tr.requires_grad_()
#muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
#end = muA.mean()
#end.backward()

In [ ]:
#wm_tr.grad.mean(dim=0)

#### Game 3

In [ ]:
# 3. Spieltag
# Was retrained on past tournaments and Game 1,2 results
neural_pickled = NBNet(len(col),30,20,4,0.3)
state = torch.load('model/model_r3.pth')
neural_pickled.load_state_dict(state["state_dict"])
#state

In [ ]:
wm = pd.read_csv(filepath_or_buffer="data/WM18.csv",delimiter=";",index_col=False).round(2)[32:48]
# impute missing past values with 0
wm.fillna({"past_resultA":0,"past_resultB":0},inplace=True)
#scale
scale(wm,state)

In [ ]:
wm_tr = torch.from_numpy(wm[col].values).float()

In [ ]:
neural_pickled.eval()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y_prob=probs,top_n=1,verbose=True)

Top-5 accuracy 62.5%

Top-3 accuracy 43.75%

Top-1 accuracy 18.75%

In [ ]:
util.multi_result(y_prob=probs,y=wm[["resultA","resultB"]].values,top_n=1,verbose=True)

In [ ]:
util.single_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)

In [ ]:
util.multi_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)

In [ ]:
#wm_tr.requires_grad_()
#wm_tr.grad.data.zero_()
#muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
#expA = muA.mean()
#expA.backward()
#wm_tr.grad.mean(dim=0)

#wm_tr.grad.data.zero_()
#muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
#expB = muB.mean()
#expB.backward()
#wm_tr.grad.mean(dim=0)

#wm_tr.grad.data.zero_()
#muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
#disB = alphaB.mean()
#disB.backward()
#wm_tr.grad.mean(dim=0)

#wm_tr.grad.data.zero_()
#muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
#disA = alphaA.mean()
#disA.backward()
#wm_tr.grad.mean(dim=0)

## Knockout Stage

In [ ]:
col=["gametype","teamA_age","teamB_age","teamA_def_val","teamB_def_val","teamA_off_val","teamB_off_val","teamA_frag","teamB_frag","past_resultA","past_resultB"]

In [ ]:
# Round of 16
neural_pickled = NBNet(len(col),30,30,4,0.25)
state = torch.load('model/model_ko16.pth')
neural_pickled.load_state_dict(state["state_dict"])
wm = pd.read_csv(filepath_or_buffer="data/WM18.csv",delimiter=";",index_col=False).round(2)[48:56]
# impute missing past values with 0
wm.fillna({"past_resultA":0,"past_resultB":0},inplace=True)
#state

In [ ]:
#scale
scale(wm,state)
wm["gametype"]=wm["gametype"].apply(map_to_round)

In [ ]:
wm_tr = torch.from_numpy(wm[col].values).float()

neural_pickled.eval()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
#print(muA.mean(),muB.mean(),alphaA.mean(),alphaB.mean())
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y_prob=probs,top_n=3,verbose=True)

In [ ]:
# Evaluate
util.multi_result(y=wm[["resultA","resultB"]].values,y_prob=probs,top_n=3,verbose=True)
util.single_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)
util.multi_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)

In [ ]:
# Viertelfinale
neural_pickled = NBNet(len(col),30,20,4,0.25)
state = torch.load('model/model_ko8.pth')
neural_pickled.load_state_dict(state["state_dict"])

wm = pd.read_csv(filepath_or_buffer="data/WM18.csv",delimiter=";",index_col=False).round(2)[56:60]
#state

In [ ]:
#scale
scale(wm,state)
wm.fillna({"past_resultA":0,"past_resultB":0},inplace=True)
wm["gametype"]=wm["gametype"].apply(map_to_round)
# Viertelfinale
wm_tr = torch.from_numpy(wm[col].values).float()

neural_pickled.eval()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
#print(muA.mean(),muB.mean(),alphaA.mean(),alphaB.mean())
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y_prob=probs,top_n=3,verbose=True)

In [ ]:
# Evaluate
util.multi_result(y=wm[["resultA","resultB"]].values,y_prob=probs,top_n=3,verbose=True)
util.single_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)
util.multi_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)

In [ ]:
wm

In [ ]:
# Halbfinale/Finals
neural_pickled = NBNet(len(col),30,20,4,0.5)
state = torch.load('model/model_ko4.pth')
neural_pickled.load_state_dict(state["state_dict"])
wm = pd.read_csv(filepath_or_buffer="data/WM18.csv",delimiter=";",index_col=False).round(2)[60:64]
#state

In [ ]:
#scale
scale(wm,state)
wm.fillna({"past_resultA":0,"past_resultB":0},inplace=True)
wm["gametype"]=wm["gametype"].apply(map_to_round)

wm_tr = torch.from_numpy(wm[col].values).float()

neural_pickled.eval()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
#print(muA.mean(),muB.mean(),alphaA.mean(),alphaB.mean())
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y_prob=probs,top_n=3,verbose=True)

In [ ]:
# Evaluate
util.multi_result(y=wm[["resultA","resultB"]].values,y_prob=probs,top_n=5,verbose=True)
util.single_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)
util.multi_tendency(y=wm[["resultA","resultB"]].values,y_prob=probs)